In [12]:
import pandas as pd
import numpy as np

## Step 1: Read the data into a pandas DataFrame
# Dep: data/depend_only.csv, FATR: data/fatr_students.csv
# df_pandas = pd.read_csv('data/fatr_students.csv')
df_pandas = pd.read_csv('data/depend_only.csv')



# Step 3: Fill missing values with the median of their respective columns




print(df_pandas[columns + ['NasaTLX']])
temp_csv_path = 'temp_cleaned_data.csv'
df_pandas.to_csv(temp_csv_path, index=False)

    Performance  Temporal Demand  Frustration Level  Mental Demand  Effort  \
0             1              3.0                  3              7       7   
1             1              2.0                  2              4       3   
2             2              3.0                  2              4       3   
3             2              2.0                  2              4       4   
4             2              2.0                  1              2       3   
..          ...              ...                ...            ...     ...   
91            5              9.0                  4              7      10   
92            8              8.0                  9              8       7   
93            2              8.0                  2              4       7   
94            5              8.0                  6              9       8   
95            4              8.0                  2              4       8   

    NasaTLX  
0       4.2  
1       2.4  
2       2.8  
3      

In [48]:
import pandas as pd
import os
import numpy as np
from pyvttbl import DataFrame
from pyvttbl.stats import Anova

##### SET STUDY VARIABLES
## Between-factors >> group on....
# Dep: Exptertise, FATR: Study
human = False
human = True

FATR = False
FATR = True


if FATR:
    print("yes fatr")
    source = os.path.join("data","fatr_students.csv")
    bfactor = "Study"
else:
    source = os.path.join("data","depend_only.csv")
    bfactor = "Expertise"


temp_csv_path = 'temp_cleaned_data.csv'

## Set DVs >> process multiple in one script, or just one at a time; 
dv_cols = ['Diagnosis Time']  
dv_cols = ['Cooper Harper']  
dv_cols = ['NasaTLX']
dv_cols = ['Decision']


# Create a pyvttbl.DataFrame from the cleaned pandas DataFrame
# Read the data into a pandas DataFrame
df = DataFrame()
df_pandas = pd.read_csv(source)


if dv_cols[0] != "NasaTLX":
    ## Drop rows with missing values in relevant columns
    # CAUTION! If dv_cols list contains multiple columns, then columns missing row values will obliterate other columns' row values
    df_pandas = df_pandas.dropna(subset=dv_cols)
    temp_csv_path = 'temp_cleaned_data.csv'
    df_pandas.to_csv(temp_csv_path, index=False)
else:
    print("create NasaTLX composit measure...")
    # Invert 'Performance' scores
    df_pandas['Performance'] = 11 - df_pandas['Performance']
    columns = ['Performance', 'Temporal Demand', 'Frustration Level', 'Mental Demand', 'Effort']
    for column in columns:
        median = df_pandas[column].median()
        df_pandas[column].fillna(median)
    # Calculate the NasaTLX composite score
    df_pandas['NasaTLX'] = df_pandas[columns].mean(axis=1)
    df_pandas.to_csv(temp_csv_path, index=False)

df.read_tbl(temp_csv_path)


for dv in dv_cols:
    print(f"Running ANOVA for: {dv}")
    
    aov = Anova()
    aov.run(df, dv, wfactors=['Scenario', 'Fault'], bfactors=[bfactor])   
    aov.truncate(test='gg')

    if human:
        ### HUMAN READABLE
        print(aov)
        print("\n\n") # this puts spaces between reports when running multiples
    else:
        ### For the Robots
        print(dict(aov))
        print("\n\n") 
    
        marginal_means = aov.get_marginal_means(['Scenario', 'Fault', bfactor])
        print(f"Marginal Means for: {dv}")
        print(marginal_means)
        print("\n\n") 

yes fatr
Running ANOVA for: Decision
('Scenario',)
('Study',)
('Fault', 'Study')
('Scenario', 'Fault', 'Study')
Decision ~ Scenario * Fault * Study

TESTS OF BETWEEN-SUBJECTS EFFECTS

Measure: Decision - No Significant Effects or Interactions


TESTS OF WITHIN SUBJECTS EFFECTS

Measure: Decision
     Source                             Type III   eps   df    MS       F      Sig.    et2_G   Obs.    SE     95% CI   lambda   Obs.  
                                           SS                                                                                   Power 
Scenario * Study   Greenhouse-Geisser      0.667     1    1   0.667    5.966   0.023   0.050     24   0.072    0.142    6.508   0.684 
-------------------------------------------------------------------------------------------------------------------------------------
Error(Scenario)    Greenhouse-Geisser      2.458     1   22   0.112                                                                   
-----------------------------

In [ ]:
from pyvttbl import DataFrame
from pyvttbl.stats import Anova

##### SET STUDY VARIABLES
## Between-factors >> group on....
# Dep: Exptertise, FATR: Study
bfactor = "Study"
bfactor = "Expertise"
output = "Robot"
output = "Human"

## Step 1: Read the data into a pandas DataFrame
# Dep: data/depend_only.csv, FATR: data/fatr_students.csv
# df_pandas = pd.read_csv('data/fatr_students.csv')
df_pandas = pd.read_csv('data/depend_only.csv')

## Step 2: Drop rows with missing values in relevant columns
# CAUTION! If dv_cols list contains multiple columns, then columns missing row values will obliterate other columns' row values
df_pandas = df_pandas.dropna(subset=dv_cols)
temp_csv_path = 'temp_cleaned_data.csv'
df_pandas.to_csv(temp_csv_path, index=False)

# Step 3: Create a pyvttbl.DataFrame from the cleaned pandas DataFrame
df = DataFrame()
df.read_tbl(temp_csv_path)

for dv in dv_cols:
    print(f"Running ANOVA for: {dv}")
    
    aov = Anova()
    aov.run(df, dv, wfactors=['Scenario', 'Fault'], bfactors=[bfactor])   
    aov.truncate(test='gg')

    if output == "Human":
        ### HUMAN READABLE
        print(aov)
        print("\n\n") # this puts spaces between reports when running multiples
    elif output == "Robot":
        ### For the Robots
        print(dict(aov))
        print("\n\n") 
    
        marginal_means = aov.get_marginal_means(['Scenario', 'Fault', bfactor])
        print(f"Marginal Means for: {dv}")
        print(marginal_means)
        print("\n\n") 